In [ ]:
#-->GUI-------------------------
from tkinter import*
from tkinter import ttk
import tkinter as tk
#-->Message dialog box----------
from tkinter import messagebox
#-->Connect Database------------
import mysql.connector
from datetime import datetime 
#-->Email validation------------
import re

#--->Attendance mark function

def attendance_mark(femail,subject):
#--> Scanning QRCode in over system------------------------------------
    import cv2
    import webbrowser
    from datetime import datetime 
    import mysql.connector
    mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="attendance")
    mycursor = mydb.cursor()

    cap = cv2.VideoCapture(0)
    detector = cv2.QRCodeDetector()

    while True:
        ret, img = cap.read()
        data, points, _ = detector.detectAndDecode(img)

        if data:
            qrdata = data
            break
        cv2.imshow("Scanner", img)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    #print(a)

    # Find the QRCode DATA------------------------------------------

    #--> Now mathing QRCode data with database-------------------------
    s = (qrdata,)
    mycursor = mydb.cursor()
    mycursor.execute("select id from student where id=%s",s)
    for i in mycursor:
        result = i[0]

    if qrdata == result:
        now = datetime.now()
        # convert to string
        date = now.strftime("%Y-%m-%d")
        sql = "select * from "+femail[0:-10]+"_"+subject
        mycursor.execute(sql)
        records = mycursor.fetchall()
        for fname_tbl in records:
            sql = "update "+femail[0:-10]+"_"+subject+" set attend=1 where s_id=%s and date=%s"
            sqlist = (result,date)
            mycursor.execute(sql,sqlist)
            mydb.commit()
    else:
        message = "Your QRcode is not valid"
        print(message)
        attendance_mark(femail,subject)
#-------------over the attendance_mark function--------------->

#-->Here validation and send the data in database---------
def login():
    femail = femailE.get()
    psw = pswE.get()
    rad = selected.get()
    print(rad)
    mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="attendance")
    mycursor = mydb.cursor()
    mycursor.execute('select * from faculty')
    records = mycursor.fetchall()
    for row in records:
        if femail==row[0] and psw==row[1]:
            getEmail=row[0]
            getPass=row[1]
            break
        else:
            getEmail=''
            getPass=''
    if femail == '' or psw == '' or rad == '':
        messagebox.showerror("Error","Fill the entry",parent=root)
    elif not(femail==getEmail and psw==getPass):
        messagebox.showerror("Error","not valid",parent=root)
    else:
        try:
            if(rad == 'attend'):
                mycursor.execute('select * from student')
                records = mycursor.fetchall()
                for std in records:
                    subject = row[3]
                    sql = "insert into "+femail[0:-10]+"_"+row[3]+"(s_id,s_name) values(%s,%s)"
                    sqlist = (std[0],std[1])
                    mycursor.execute(sql,sqlist)
                    mydb.commit()
                messagebox.showinfo("Information","Login Successfully! this is for attend the lecture", parent=root)
                #attendance_mark(femail,subject)
            else:
                messagebox.showinfo("Information","Login Successfully! But this is for use only", parent=root)
        except Exception as es:
            messagebox.showerror("Error", f"Error Due to: {str(es)}", parent=root)
#--->Login function over------------

#-->Main Code start-----------------
root = Tk()
root.title("Login")
root.geometry("500x500")

#-->Heading Registarion-------------
heading = Label(root, text="Login", font=("times new roman",30))
heading.place(x=150,y=50)

#-->Faculty Email-Id----------------
femail = Label(root, text="Email-Address:", font=("times new roman",15))
femail.place(x=50,y=130)
femailE = Entry(root,font=(15))
femailE.place(x=200,y=132)

#-->Faculty Password-------
psw = Label(root, text="Password:", font=("times new roman",15))
psw.place(x=50,y=180)
pswE = Entry(root,show="●",font=(15))
pswE.place(x=200,y=182)

#-->Radio Button For insert the value or not
selected = tk.StringVar()
r1 = ttk.Radiobutton(root, text="Just for use only",value='use', variable=selected).place(x=50,y=220)
r2 = ttk.Radiobutton(root, text="Attend the lecture",value='attend', variable=selected).place(x=200,y=220)
#-->Submit Event"
submit = Button(root, text="Submit", command=login)
submit.place(x=150,y=250) 

root.mainloop()
